<a href="https://colab.research.google.com/github/harin/cudsi-capstone-2019/blob/master/embedding/generate_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==2.2 spacy-transformers
!python -m spacy download en_trf_bertbaseuncased_lg

  Using cached https://files.pythonhosted.org/packages/1c/96/10881bc82d05664c40a38cf7c645e6ab1877e5808ef4a3684a99cd1445a4/spacy-2.2.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/4c/60/cbe7c1ca0d05e9861375fc9fe73f5c6828666bb994e193a80bce554c4004/thinc-7.1.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: spacy-transformers 0.5.1 has requirement spacy<2.3.0,>=2.2.1, but you'll have spacy 2.2.0 which is incompatible.
ERROR: en-trf-bertbaseuncased-lg 2.2.0 has requirement spacy>=2.2.1, but you'll have spacy 2.2.0 which is incompatible.
  Found existing installation: thinc 7.3.1
    Uninstalling thinc-7.3.1:
      Successfully uninstalled thinc-7.3.1
  Found existing installation: spacy 2.2.3
    Uninstalling spacy-2.2.3:
      Successfully uninstalled spacy-2.2.3
     |████████████████████████████████| 10.4MB 6.4MB/s 
     |████████████████████████████████| 2.2MB 45.6MB/s 
  Found existing installation: thinc 7.1.1
    Uninstalling thinc-7.1.1:
      Suc

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import spacy
import torch

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

nlp = spacy.load("en_trf_bertbaseuncased_lg")

In [16]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
STOPWORDS = set(stopwords.words('english'))

def clean_text_lemma(text):
    """
        text: a paragrapy
        
        return: modified initial string
    """
    text = re.sub(r"[^\w\s]", '', text) ## Remove all non-word characters (everything except numbers and letters)
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    text_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in text_list]) # lemmatization
    #text_list2 = nltk.word_tokenize(lemmatized_output)
    #stemmed_output = ' '.join([porter.stem(w) for w in text_list2]) #stemming
    return lemmatized_output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
!ls "/content/gdrive/My Drive/DS_Capstone/data/"


clustering  raw  sample_articles.csv


In [0]:
import pandas as pd
def read_article_df(file):
    articles_dt = file.read().split('\n')[:-1]
    pubId, canonicalUrl,firstScrape,title,text,lang_reliability = [],[],[],[],[],[]
    lang_iso = []
    for article in articles_dt:    
        row = article.split('\t')
        pubId.append(row[0])
        canonicalUrl.append(row[4])
        firstScrape.append(row[5])
        lang_iso.append(row[6])
        lang_reliability.append(row[7])
        title.append(row[8])
        text.append(row[9])

    articles_df = pd.DataFrame()
    articles_df['pubId'] = pubId
    articles_df['canonicalUrl'] = canonicalUrl
    articles_df['firstScrape'] = firstScrape
    articles_df['title'] = title
    articles_df['text'] = text
    articles_df['lang_reliability'] = lang_reliability
    articles_df['lang_iso'] = lang_iso
    return articles_df

In [0]:
sample_df = pd.read_csv("/content/gdrive/My Drive/DS_Capstone/data/sample_articles.csv", index_col=0)

In [8]:
sample_df.all_text = sample_df.title + sample_df.text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [17]:
sample_df.clean_text = sample_df.all_text.apply(clean_text_lemma)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [21]:
from tqdm import tqdm

embeddings = []
for doc in tqdm(nlp.pipe(sample_df.clean_text)):
  embeddings.append(doc.tensor.sum(axis=0).tolist())

10140it [23:00,  7.34it/s]


In [0]:
df = pd.DataFrame(embeddings, index=sample_df.index)

In [0]:
df.to_csv("/content/gdrive/My Drive/DS_Capstone/data/sample_bert.csv")